# Módulo 6. Optimización de hiper-parámetros

¡Bienvenidos al sexto módulo! Ahora que ya sabemos lo que son las redes neuronales, las redes convolucionales y todos sus parámetros y entresijos, vamos a ver cómo podemos optimizarlas al máximo.


## 1. Grid search

El grid search es el método más sencillo que existe para encontrar los mejores parámetros dentro de un conjunto. Esencialmente se trata de fuerza bruta. Vamos a ver un ejemplo sencillo:

Suponed que tenemos nuestra anterior red y queremos ver qué parámetros son los mejores. Tenemos:

* dropout: puede variar de 0 a 0.5 en intervalos 0.1
* learning rate: puede variar de 0.1 a 0.001 en intervalos de x10
* número de filtros: puede variar de 64 a 256 en intervalos de 64
* tamaños del filtro: puede variar desde 3 hasta 7 de 2 en 2 (filtros cuadrados siempre)

Pues bien, lo que el grid search haría es los iguiente:

In [2]:
# Definimos nuestra grid
dropout = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
lr = [0.1, 0.01, 0.001]
nf = [64, 128, 256]
fs = [3, 5, 7]
print(f"Nb experimentos: {len(dropout)*len(lr)*len(nf)*len(fs)}")
# creamos una variable para guardar nuestras accuracies
log_accuracies = []

# para hacer esta prueba definimos una función que sustituye a nuestra red y
# asigna un valor aleatorio cada vez
# si lo hiciésemos con nuestra red de verdad, tardaría demasiado como para verlo
# en clase
from random import uniform
def dummy_net(d, lr, nf, fs):
  print('Ejecutando la red con d={}, lr={}, nf={}, fs={}'.format(d, lr, nf, fs))
  return uniform(0,1)

# contador
i = 1

# empezamos el grid search

for d in dropout:
  for lr_ in lr:
    for nf_ in nf:
      for fs_ in fs:
        result_net = dummy_net(d, lr_, nf_, fs_)
        log_accuracies.append(result_net)


Nb experimentos: 162
Ejecutando la red con d=0.0, lr=0.1, nf=64, fs=3
Ejecutando la red con d=0.0, lr=0.1, nf=64, fs=5
Ejecutando la red con d=0.0, lr=0.1, nf=64, fs=7
Ejecutando la red con d=0.0, lr=0.1, nf=128, fs=3
Ejecutando la red con d=0.0, lr=0.1, nf=128, fs=5
Ejecutando la red con d=0.0, lr=0.1, nf=128, fs=7
Ejecutando la red con d=0.0, lr=0.1, nf=256, fs=3
Ejecutando la red con d=0.0, lr=0.1, nf=256, fs=5
Ejecutando la red con d=0.0, lr=0.1, nf=256, fs=7
Ejecutando la red con d=0.0, lr=0.01, nf=64, fs=3
Ejecutando la red con d=0.0, lr=0.01, nf=64, fs=5
Ejecutando la red con d=0.0, lr=0.01, nf=64, fs=7
Ejecutando la red con d=0.0, lr=0.01, nf=128, fs=3
Ejecutando la red con d=0.0, lr=0.01, nf=128, fs=5
Ejecutando la red con d=0.0, lr=0.01, nf=128, fs=7
Ejecutando la red con d=0.0, lr=0.01, nf=256, fs=3
Ejecutando la red con d=0.0, lr=0.01, nf=256, fs=5
Ejecutando la red con d=0.0, lr=0.01, nf=256, fs=7
Ejecutando la red con d=0.0, lr=0.001, nf=64, fs=3
Ejecutando la red con d=0

In [3]:
# y el mejor resultado sería
import numpy as np
idx_max = np.nonzero(log_accuracies == np.max(log_accuracies))
print(idx_max)

print('Best execution: {}. Accuracy: {}'.format(idx_max[0][0], log_accuracies[idx_max[0][0]]))

(array([76]),)
Best execution: 76. Accuracy: 0.9991321288966578


Y con esto sabríamos cuál es la mejor configuración de nuestra red. Mucho mejor que andar cambiando parámetros a mano, ¿verdad?

¿Cuál es el problema de este método? Pues que tenemos 6 x 4 x 4 x 3 ejecuciones de nuestra red, lo que hacen un total de 288 ejecuciones, a un mínimo de 10 minutos por ejecución, son 48 horas, ¡¡¡2 días!!!

Si queréis comprobarlo no tenéis más que meter dentro de dummy_net la arquitectura de la red y ejecutarlo ;-)

Vamos a hacer una prueba con una red muy sencillita para que veáis qué tal funciona:

In [1]:
# importamos los paquetes necesarios
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# cargamos los datos y pasamos de vector a imagen
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# normalizamos
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convertimos las etiquetas a one-hot encoding
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

def net(do, lr, nf, fs):
  model = Sequential()
  model.add(Conv2D(nf, kernel_size=(fs, fs), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(do))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(do))
  model.add(Dense(n_classes, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=128, epochs=1, verbose=0)
  score = model.evaluate(x_test, y_test, verbose=0)
  print('Red con d={}, lr={}, nf={}, fs={}. Loss: {}. Acc: {}.'.format(d, lr, nf, fs, score[0], score[1]))
  return score[0]

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [2]:
# definimos nuestra grid
dropout = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
lr = [0.1, 0.01, 0.001]
nf = [64, 128, 256]
fs = [3, 5, 7]

# creamos una variable para guardar nuestras accuracies
log_accuracies = []

# contador
i = 1

# empezamos el grid search
for d in dropout:
  for lr_ in lr:
    for nf_ in nf:
      for fs_ in fs:
        result_net = net(d, lr_, nf_, fs_)
        log_accuracies.append(result_net)
        print('Experimento {}/{}'.format(i, len(dropout)*len(lr)*len(nf)*len(fs)))
        i += 1

# y el mejor resultado sería
import numpy as np
idx_max = np.nonzero(log_accuracies == np.min(log_accuracies))
print(idx_max)
print('Best execution: {}. Accuracy: {}'.format(idx_max[0][0], log_accuracies[idx_max[0][0]]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Red con d=0.0, lr=0.1, nf=64, fs=3. Loss: 0.27455753087997437. Acc: 0.9265000224113464.
Experimento 1/162
Red con d=0.0, lr=0.1, nf=64, fs=5. Loss: 2.3080334663391113. Acc: 0.11349999904632568.
Experimento 2/162
Red con d=0.0, lr=0.1, nf=64, fs=7. Loss: 0.33445125818252563. Acc: 0.9067000150680542.
Experimento 3/162
Red con d=0.0, lr=0.1, nf=128, fs=3. Loss: 2.3088479042053223. Acc: 0.11349999904632568.
Experimento 4/162
Red con d=0.0, lr=0.1, nf=128, fs=5. Loss: 2.307349681854248. Acc: 0.10279999673366547.
Experimento 5/162
Red con d=0.0, lr=0.1, nf=128, fs=7. Loss: 0.2873222827911377. Acc: 0.9230999946594238.
Experimento 6/162


KeyboardInterrupt: 

In [ ]:
# definimos nuestra grid con mejores valores
##Code##
# creamos una variable para guardar nuestras accuracies
log_accuracies = []

# contador
i = 1

# empezamos el grid search
##Code##

# y el mejor resultado sería
import numpy as np
idx_max = ##Code##
print(idx_max)

print('Best execution: {}. Accuracy: {}'.format(##Code##))

Bueno, no está mal, ¿verdad?

Aunque estaría genial que hubiesen métodos más rápidos o usando algo de heurística, ¿no? En vez de fuerza bruta...

Pues estáis de suerte! Existen varios métodos de este tipo:

* Spearmint (Python)
* BayesOpt (C++ with Python and Matlab/Octave interfaces)
* hyperopt (Python)
* SMAC (Java)
* REMBO (Matlab)
* MOE (C++/Python)

+INFO: http://fastml.com/optimizing-hyperparams-with-hyperopt/

Y hoy vamos a ver **hyperopt**

## 1.2 Hyper-opt

Otra opción es utilizar hyperopt (Hyperopt: Distributed Asynchronous Hyper-parameter Optimization, https://github.com/hyperopt/hyperopt).

Hyperopt es una librería escrita en Python que permite optimizar funciones de una forma rápida fijándose más en los valores que más probablemente van a dar una buena solución.

Actualmente tiene dos algoritmos implementados para hacer esto:

* Random Search
* Tree of Parzen Estimators (TPE)

Además, se pueden ejecutar en serie o en paralelo, haciendo uso de MongoDB.

Vamos a ver un ejemplo de cómo utilizarlo.

Vamos a encontrar el mínimo de $x^2$:
(<a href="https://www.google.es/search?source=hp&ei=2ds5XO_jG6y_lwTvxp2QBQ&q=x**2&btnK=Google+Search&oq=x**2&gs_l=psy-ab.3...716.1584..1916...0.0..0.301.1084.1j0j3j1......0....1..gws-wiz.....0..0i131j0.cgRgMW95cQk">click aqui para ver la funcion</a>)


In [1]:
from hyperopt import fmin, tpe, hp
# con 10 iteraciones

best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform("x", -10, 10),
            algo=tpe.suggest,
            max_evals=10)

print(best)

100%|██████████| 10/10 [00:00<00:00, 774.34trial/s, best loss: 0.0914586592950828]
{'x': np.float64(0.3024213274474583)}


In [2]:
# con 100 iteraciones
best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform("x", -10, 10),
            algo=tpe.suggest,
            max_evals=100)
print(best)

100%|██████████| 100/100 [00:00<00:00, 540.98trial/s, best loss: 1.4650031466190746e-05]
{'x': np.float64(-0.0038275359523054445)}


In [3]:
# con 1000 iteraciones
best = fmin(fn=lambda x: x ** 2,
            space=hp.uniform("x", -10, 10),
            algo=tpe.suggest,
            max_evals=1000)

print(best)

100%|██████████| 1000/1000 [00:04<00:00, 216.51trial/s, best loss: 1.985167812026748e-06]
{'x': np.float64(-0.001408959833361742)}


Y ahora uno más complejo con redes neuronales:

In [ ]:
# instalamos los paquetes necesarios
# !pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
# !pip install hyperopt

     |████████████████████████████████| 1.3 MB 4.2 MB/s 
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.3
    Uninstalling networkx-2.6.3:
      Successfully uninstalled networkx-2.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.18.3 requires networkx>=2.0, but you have networkx 1.11 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
from re import X
# imports necesarios
import sys
import time
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10

SEED = 42

#Cargamos y normalziamos los datos
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
validation_split = 0.1
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split)

X_train = X_train.astype("float32") / 255.
X_val = X_val.astype("float32") / 255.
X_test = X_test.astype("float32") / 255.

# convertimos las etiquetas a one-hot encoding
n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense

space =  {
  "n_filters_conv": hp.choice("n_filters_conv", [32, 64, 128]),
  "dropout": hp.uniform("dropout", 0.0, 0.5),
  "neurons_dense": hp.choice("neurons_dense", [256, 512, 1024])
}

# nos definimos unos callbacks para que la red pare cuando no vea mejora

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor="val_loss", min_delta=0.001, patience=2, verbose=0, mode="auto")]
  return callbacks

def mi_cnn(pars: dict):
  print ('Parameters: ', pars)
  model = Sequential()
  # Primer bloque convolucional
  model.add(Conv2D(pars["n_filters_conv"], kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars["dropout"]))
  # Segundo bloque convolucional
  model.add(Conv2D(pars["n_filters_conv"], kernel_size=(3, 3), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars["dropout"]))
  # Tercer bloque convolucional
  model.add(Conv2D(pars["n_filters_conv"], kernel_size=(3, 3), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars["dropout"]))
  # Bloque clasificador
  model.add(Flatten())
  model.add(Dense(pars["neurons_dense"], activation="relu"))
  model.add(Dropout(pars["dropout"]))
  model.add(Dense(10, activation="softmax"))
  # Compilamos el modelo
  model.compile(loss="categorical_crossentropy",
                optimizer=Adam(0.001),
                metrics=["accuracy"]
  )

  # Entrenamos el modelo
  history = model.fit(
      X_train,
      y_train,
      batch_size=128,
      shuffle=True,
      epochs=10,
      verbose=0,
      validation_data=(X_val, y_val),
      callbacks=get_callbacks(pars)
  )

  best_epoch_loss = np.argmin(history.history["val_loss"])
  best_val_loss = np.min(history.history["val_loss"])
  best_val_acc = np.max(history.history["val_accuracy"])

  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()

  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 20s 0us/step


In [3]:
# ejecutamos la optimización
trials = Trials()
best = fmin(fn=mi_cnn,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)
print(best)

Parameters: 
{'dropout': 0.07713717705812995, 'n_filters_conv': 128, 'neurons_dense': 256}
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 8 - val acc: 0.7437999844551086 - val loss: 0.759219765663147
Parameters: 
{'dropout': 0.20686865758211703, 'n_filters_conv': 32, 'neurons_dense': 512}
Epoch 9 - val acc: 0.6597999930381775 - val loss: 0.995554506778717
Parameters: 
{'dropout': 0.38680289814691166, 'n_filters_conv': 128, 'neurons_dense': 256}
Epoch 9 - val acc: 0.7221999764442444 - val loss: 0.8254798650741577
Parameters: 
{'dropout': 0.2701021248334779, 'n_filters_conv': 64, 'neurons_dense': 1024}
Epoch 9 - val acc: 0.7092000246047974 - val loss: 0.8704171180725098
Parameters: 
{'dropout': 0.2562916164374491, 'n_filters_conv': 64, 'neurons_dense': 256}
Epoch 9 - val acc: 0.7059999704360962 - val loss: 0.8632286190986633
Parameters: 
{'dropout': 0.40386151523479213, 'n_filters_conv': 128, 'neurons_dense': 1024}
Epoch 9 - val acc: 0.7411999702453613 - val loss: 0.7853220105171204
Parameters: 
{'dropout': 0.36142119952176927, 'n_filters_conv': 32, 'neurons_dense': 1024}
Epoch 8 - val acc: 0.5979999899864197 - val l

In [4]:
trials.trials

[{'state': 2,
  'tid': 0,
  'spec': None,
  'result': {'loss': 0.759219765663147,
   'best_epoch': np.int64(8),
   'eval_time': 1742497069.1794314,
   'status': 'ok',
   'model': <Sequential name=sequential, built=True>,
   'history': <keras.src.callbacks.history.History at 0x7df2dbec1110>},
  'misc': {'tid': 0,
   'cmd': ('domain_attachment', 'FMinIter_Domain'),
   'workdir': None,
   'idxs': {'dropout': [np.int64(0)],
    'n_filters_conv': [np.int64(0)],
    'neurons_dense': [np.int64(0)]},
   'vals': {'dropout': [np.float64(0.07713717705812995)],
    'n_filters_conv': [np.int64(2)],
    'neurons_dense': [np.int64(0)]}},
  'exp_key': None,
  'owner': None,
  'version': 0,
  'book_time': datetime.datetime(2025, 3, 20, 18, 56, 44, 73000),
  'refresh_time': datetime.datetime(2025, 3, 20, 18, 57, 49, 179000)},
 {'state': 2,
  'tid': 1,
  'spec': None,
  'result': {'loss': 0.995554506778717,
   'best_epoch': np.int64(9),
   'eval_time': 1742497099.712795,
   'status': 'ok',
   'model': <S

¿Por qué dice que best_epoch=4? Porque history está indexado en 0.

In [5]:
trials.trials[6]

{'state': 2,
 'tid': 6,
 'spec': None,
 'result': {'loss': 1.1624877452850342,
  'best_epoch': np.int64(8),
  'eval_time': 1742497329.4981873,
  'status': 'ok',
  'model': <Sequential name=sequential_6, built=True>,
  'history': <keras.src.callbacks.history.History at 0x7df2ce02e7d0>},
 'misc': {'tid': 6,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'dropout': [np.int64(6)],
   'n_filters_conv': [np.int64(6)],
   'neurons_dense': [np.int64(6)]},
  'vals': {'dropout': [np.float64(0.36142119952176927)],
   'n_filters_conv': [np.int64(0)],
   'neurons_dense': [np.int64(2)]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 3, 20, 19, 1, 40, 51000),
 'refresh_time': datetime.datetime(2025, 3, 20, 19, 2, 9, 498000)}

In [6]:
trials.results

[{'loss': 0.759219765663147,
  'best_epoch': np.int64(8),
  'eval_time': 1742497069.1794314,
  'status': 'ok',
  'model': <Sequential name=sequential, built=True>,
  'history': <keras.src.callbacks.history.History at 0x7df2dbec1110>},
 {'loss': 0.995554506778717,
  'best_epoch': np.int64(9),
  'eval_time': 1742497099.712795,
  'status': 'ok',
  'model': <Sequential name=sequential_1, built=True>,
  'history': <keras.src.callbacks.history.History at 0x7df2d2e1b2d0>},
 {'loss': 0.8254798650741577,
  'best_epoch': np.int64(9),
  'eval_time': 1742497158.8832984,
  'status': 'ok',
  'model': <Sequential name=sequential_2, built=True>,
  'history': <keras.src.callbacks.history.History at 0x7df2d2d06010>},
 {'loss': 0.8704171180725098,
  'best_epoch': np.int64(9),
  'eval_time': 1742497195.6794398,
  'status': 'ok',
  'model': <Sequential name=sequential_3, built=True>,
  'history': <keras.src.callbacks.history.History at 0x7df2d2c2e010>},
 {'loss': 0.8632286190986633,
  'best_epoch': np.int6

In [7]:
trials.losses()

[0.759219765663147,
 0.995554506778717,
 0.8254798650741577,
 0.8704171180725098,
 0.8632286190986633,
 0.7853220105171204,
 1.1624877452850342,
 0.7692969441413879,
 0.9629977345466614,
 0.8574033379554749]

¿Qué os parece?  Así, podéis dejar vuestra configuración e iros a hacer algo más útil que andar variando parámetros hasta que encontréis la configuración adecuada.

**¡Mejor que lo hagan por nosotros y que nos la encontremos automáticamente seleccionada!**

Pero no tenemos por qué quedarnos aquí, podemos también variar el número de capas o si queremos conexiones residuales, por ejemplo! Sí, esto significa que podemos variar la **arquitectura** también.

Aquí tenéis un ejemplo muy completo: https://github.com/Vooban/Hyperopt-Keras-CNN-CIFAR-100

Y aquí otro que quizá os parezca interesante:

<img src="http://cdn.lizamayliza.com/storage/cache/images/003/636/Daffy-Duck-Money-eyes-feature,xlarge.1480369578.jpg" border="0" height="200">

https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-hyperparameters-optimization-cb2b4a29b8ee

## 1.3 Algoritmos genéticos

Quizás estéis pensando en que tras haber visto esto no necesitáis saber nada más, ¡ya habéis encontrado algo que es la repera! Pues atentos porque lo que viene, pese a ser algo muy sencillo, es potentísimo, y os va encantar. Sí, estoy hablando de **los algoritmos genéticos**.

En esencia, los algoritmos genéticos son un método de búsqueda meta-heurística inspirados en la evolución natural. Pertenecen a los algoritmos evolutivos, concretamente a los Algoritmos de búsqueda aleatoria guiada (Guided Random Search algorithms (Evolutionary Alg.)).

Esto que os puede parecer chino, es muy sencillo. Vamos a entenderlos con un ejemplo:

<img src="https://image.ibb.co/cJcQYJ/ga_problem.png" alt="ga_problem" border="0">

Imaginad que tenemos un puzzle, y nos queda solo una pieza por encajar. Lo que pasa es que este puzzle es muy especial, porque nos permite fabricarnos nuestras propias piezas. Para ello, disponemos de varios mecanismos:

* **Combinar** partes de piezas (**crossover** o recombinación).
* **Modificar** determinadas partes de esas piezas (**mutación**).
* **Escoger** las mejores piezas de todas las que hemos hecho, para a partir de ellas, construir nuevas que sean mejores (**selección**).

Entonces imaginaos que decidimos cortar 10 trozos de carton, que son nuestras 10 piezas iniciales con las que vamos a probar a ver si alguna encaja perfectamnte. Las probamos todas, y de esas 10, 5 encajan más o menos. Así que seleccionamos esas 5 y fabricamos nuevas a partir de ellas mediante los mecanismos explicados arriba:

* De las 5 seleccionadas, sacamos 5 más combinando partes de dos de esas 5 originales escogiéndolas aleatoriamente
* De las 5 originales, y las nuevas 5 que nos hemos creado, sacamos 5 más modificando ligeramente una de las puntas de la pieza

Ahora resulta que tenemos 15 piezas, y nosotros queremos tener siempre 10, porque si no a la 5 vez que hiciéramos esto tendríamos una barbaridad de piezas!! Así que:

* Probamos nuestras 15 piezas y nos quedamos con la que mejor encaja, y luego 9 escogidas al azar

Pues ya está, ¡eso es lo que hace un algoritmo genético! ¡Ya sabéis cómo funcionan! Sencillo, ¿verdad? Pues no os hacéis una idea de lo potentes que son :-)

Vamos a verlo un poco más concreto siguiendo con nuestro ejemplo:

<img src="https://image.ibb.co/b7ySfy/geneticos_puzzle.png" alt="geneticos_puzzle" border="0">

Como podéis ver:

* Cada pieza de nuestro conjunto de piezas (población) es un **cromosoma**
* Cada parte de nuestra pieza es un **gen**, por tanto, nuestro cromosoma tiene 4 genes
* Los posibles valores o configuraciones que puede tener cada gen se llama **alelo**

Esto es exactamente igual que en biología! No os dije que estaban inspirados en la evolución natural? :-)

Vale, pues vamos a ir relacionando estas palabrejas con nuestro ejemplo:

* Necesitamos encontrar la pieza correcta para nuestro hueco del puzzle
* Tenemos un conjunto inicial de piezas (**población**) que pueden encajar bien o no, no lo sabemos
* Comprobamos cómo de bien encajan estas piezas (usando la **función de fitness**)
* Si ninguna de las piezas encaja como nos gustaría, modificamos las piezas (con los operadores: **crossover** y **mutación**)
* Comprobamos como de bien encajan las piezas recien creadas (**función de fitness**)
* Seleccionamos las piezas que queremos aguantar para la próxima iteración (**selección**)

* Y volvemos a empezar! así, hasta que encontremos una pieza que encaje con la precisión que nosotros queremos

Venga, vamos a ponernos un poco más serios. Veamos el pseudo-algoritmo:

**START**

Generate the initial population

Compute fitness

**REPEAT**
    
    Selection
    
    Crossover
    
    Mutation

    Compute fitness

**UNTIL** population has converged

**STOP**

¿Se entiende, no? ¡Es exactamente lo mismo que los pasos que acabamos de hablar con el puzzle!

<img src="https://image.ibb.co/kQu7Ed/ga_workflow.png" alt="ga_workflow" border="0" width="600">

Vale, y ahora llega lo interesante, ¿cómo funcionan realmente?

Tenemos que entender varios conceptos:

* Cómo se inicializa nuestra población
* Cómo funciona el crossover
* Cómo funciona la mutación
* Cómo funciona la seleccion
* Cómo podemos definir nuestra función de fitness

¿Preparados? ¡Vamos allá!

Lo primero es entender que cuando tenemos un problema en el mundo real y queremos solucionarlo en un ordenador, necesitamos **codificarlo** para que el ordenador lo entienda.

Así, por ejemplo:

* En el mundo real, el cromosoma es la pieza del puzzle. En el ordenador, el cromosoma es un vector con 4 valores (uno para indicar el tamaño de cada punta, donde positivo significa punta, negativo significa hueco hacia dentro de la pieza)

Esto es a lo que se llama la codificación.

Una vez sabido esto, vamos a ver cómo funcionan los operadores. Antes de nada, debéis saber que xisten muchos tipos de crossover, mutación y selección, pero aquí vamos a ver solo los más sencillos por temas de tiempo.

Si estáis interesados en conocerlos más profundamente, en internet hay muchísimo material disponible. Podéis empezar aquí: https://www.tutorialspoint.com/genetic_algorithms/index.htm


<img src="https://image.ibb.co/hEwVHd/ga_operators.png" alt="ga_operators" border="0">

**El crossover de un único punto (single-point crossover)**

Nuestro cromosoma es la pieza de puzzle, que tiene los 4 genes que veis en la imagen. Pues el crossover simple sencillamente escoge un punto aleatoriamente de los 4 genes, y combina las partes en nuevos cromosomas, como veis en la imagen.

Es importante que entendáis que el crossover **produce nuevos cromosomas**, puesto que tenemos los originales y los **recombinados**.

**La mutación uniforme**

La mutación uniforme consiste en que para cada cromosoma, lanzamos una moneda al aire. Si sale cara, modificamos un gen escogido aleatoriamente. ¿Qué valor le asignamos? Uno aleatorio dentro del rango que permite dicho gen.

**La selección**

Para la selección lo que se suele hacer es usar las fitness de los cromosomas (también llamados posibles **soluciones**). En este caso, vamos a ver la Stochastic Universal Sampling, que consiste en que construímos una gráfica de tipo tarta donde cada cromosoma ocupa un espacio que corresponde con su fitness. Después, establecemos N puntos fijos alrededor de la "tarta", donde N son el número de cromomsomas que queremos **seleccionar**. Después, se "gira la tarta", como si fuese la ruleta de la suerte, y los cromosomas a los que apuntan los puntos fijos son los seleccionados y pasan a la siguiente iteración.

Si os fijáis, **los cromosomas no están ordenados de mayor a menor fitness**!!

Esto es importante, puesto que si no, las probabilidades de seleccionar un cromosoma con una fitness alta y otro con una fitness baja serían más altas que de seleccionar dos con la fitness alta, ya que al estar los puntos de selección uno enfrente del otro, sería muy complicado seleccionar dos cromosomas con fitness parecidas.

Este operador tiene varias formas de funcionar. Siguiendo con nuestro ejemplo de población de 10 cromosomas, las formas son:

* Seleccionamos $N=10$ cromosomas, es decir, sustituimos la anterior población por una completamente nueva
* Seleccionamos $N=n$ cromosomas, donde n<10. Es decir, sustituímos solo una parte de los crosomomas antiguos. El resto siguen jugando ;-D

Vale, pues si seleccionamos los 10 está claro, pero si seleccionamos $n$, cómo elegimos cuales quitamos?

Pues las dos formas más comunes son:

* Quitamos los cromosomas más antiguos
* Quitamos los cromosomas con peor fitness

Ya por último, hay veces que seleccionamos al mejor cromosoma (o a los $k$ mejors) para que pase si o si a la siguiente iteración, es decir, hay *elitismo*. Hay que ir con cuidado con esto, puesto que aunque a priori parezca que el elitismo es lo mejor y que solo deberíamos quedarnos con los mejores, si lo hiciésemos nos estaríamos cargando una de las mayores virtudes de los genéticos: **que son capaces de escapar a mínimos locales!!!**

Fijaos, aquí podéis ver en plena acción a un genético intentando decidir cual es la mejor configuración para un vehículo de 2 ruedas: http://rednuht.org/genetic_cars_2/



Vale, ¿pues ahora qué tal si implementamos un par de ejemplos nosotros?





In [2]:
# Ejemplo simple de un GA donde tenemos que encontrar N números que sumen X
# https://lethain.com/genetic-algorithms-cool-name-damn-simple

from random import randint, random
from operator import add
from functools import reduce
import numpy as np

def individual(length, min, max):
    # creamos un individuo
    return [randint(min, max) for x in range(length)]

def population(count, length, min, max):
    # creamos nuestra población
    # count: numero de invidiuos en la población
    # length: número de valores por individuo
    # min: minimo permitido para cada valor del individuo
    # max: maximo permitido para cada valor del individuo
    return [individual(length, min, max) for x in range(count)]

def fitness(individual, target):
    # calculamos la fitness de un individuo: más bajo es mejor
    sum = reduce(add, individual, 0)
    return abs(target-sum)

def grade(pop, target):
    # calculamos la media de la población entera
    summed = reduce(add, (fitness(x, target) for x in pop), 0)
    return summed / (len(pop) * 1.0)

def find_best_solution(pop, target):
    # encuentra la mejor solución en la población actual y la imprime
    res = [fitness(x, target) for x in pop]
    res_min = np.min(res)
    res_min_idx = np.where(res == res_min)[0]
    for n in res_min_idx:
        print('Individual: ', n, 'Valores: ', *pop[n], ' Result: ', np.sum(pop[n]), 'Target; ', target)
    return res_min

def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [(fitness(x, target), x) for x in pop]
    graded = [x[1] for x in sorted(graded)]
    retain_length = int(len(graded) * retain)
    parents = graded[:retain_length]

    # añadimos individuos aleatoriamente para promover la diversidad genética
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)

    # mutamos algunos
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            individual[pos_to_mutate] = randint(i_min, i_max)

    # reproducimos (crossover) nuestros cromosomas (individuals, soluciones)
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = round(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)
    parents.extend(children)
    return parents

# Ejecutamos el GA

generations = 20
target = 108
p_count = 20
i_length = 5
i_min = 0
i_max = 100
error_accepted = 1

# construimos la primera generacion
p = population(p_count, i_length, i_min, i_max)
# calculamos su ajuste a la tarea
fitness_history = [grade(p, target),]
# el mejor individup de la población inicial es
find_best_solution(p, target)

for i in range(generations):
  p = evolve(p, target)
  res = grade(p, target)
  fitness_history.append(res)
  res_min = find_best_solution(p, target)
  print(f"Generacion {i + 1}, Fitness medio de todos los individuos: {res}")
  if res_min < error_accepted:
    break

Individual:  13 Valores:  92 44 12 10 13  Result:  171 Target;  108
Individual:  6 Valores:  79 14 12 10 13  Result:  128 Target;  108
Generacion 1, Fitness medio de todos los individuos: 96.7
Individual:  8 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  18 Valores:  18 40 12 10 13  Result:  93 Target;  108
Generacion 2, Fitness medio de todos los individuos: 65.25
Individual:  0 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  1 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  7 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  9 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  19 Valores:  18 40 12 10 13  Result:  93 Target;  108
Generacion 3, Fitness medio de todos los individuos: 44.3
Individual:  0 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  1 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  2 Valores:  18 40 12 10 13  Result:  93 Target;  108
Individual:  3 Valores:  18 

In [ ]:
# Ejemplo un poco más complejo de un GA que tiene que encontrar la cadena 'Hello Word!'
# http://www.obitko.com/tutorials/genetic-algorithms/ga-basic-description.php

import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

class GeneticFunctions(object):
    def probability_crossover(self):
        # returns rate of occur crossover(0.0-1.0)
        return 1.0

    def probability_mutation(self):
        # returns rate of occur mutation(0.0-1.0)
        return 0.0

    def initial(self):
        # returns list of initial population
        return []

    def fitness(self, chromosome):
        # returns domain fitness value of chromosome
        return len(chromosome)

    def check_stop(self, fits_populations):
        # stop run if returns True
        # - fits_populations: list of (fitness_value, chromosome)
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class GuessText(GeneticFunctions):
        def __init__(self, target_text,
                     limit=200, size=400,
                     prob_crossover=0.9, prob_mutation=0.2):
            self.target = self.text2chromo(target_text)
            self.counter = 0

            self.limit = limit
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return -sum(abs(c - t) for c, t in zip(chromo, self.target))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 10 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
                print(
                    "[G %3d] score=(%4d, %4d, %4d): %r" %
                    (self.counter, best, ave, worst,
                     self.chromo2text(best_match)))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-5, 5)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return [random.randint(1, 255) for i in range(len(self.target))]
        pass

    GeneticAlgorithm(GuessText("Hello World!")).run()
    pass